In [11]:
import numpy as np
import pandas as pd
import pandas_profiling as pp
import seaborn as sns

# from sklearn.model_selection import train_test_split
# from sklearn import linear_model
# from sklearn import tree
# from sklearn import ensemble
# from sklearn import metrics

import json
import requests
%matplotlib inline

### Read all data and generate report with dataset overview

In [2]:
full_data = pd.read_csv("../data/test3.csv")
pp.ProfileReport(full_data).to_file(outputfile="./profile-test-full.html")

### Load Dataset
- Manually select features based on previous report analysis
- Load data, both to_predict and training datasets

In [8]:
feature_cols = ["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_REDACAO", "TX_RESPOSTAS_MT"]
data_to_predict = pd.read_csv("../data/test3.csv",
                         usecols=["NU_INSCRICAO"]+feature_cols)
# pp.ProfileReport(data_to_predict).to_file(outputfile="./profile-test.html")
data_train = pd.read_csv("../data/train.csv", usecols=feature_cols).dropna()
# pp.ProfileReport(data_train).to_file(outputfile="./profile-train.html")

In [10]:
data_to_predict.TX_RESPOSTAS_MT[0]

'BCEDADCCAECEABABDCEBABEDAAECCDDBDBABDADB'

### Random Strategy

In [143]:
options = ['A', 'B', 'C', 'D', 'E']
def generate_answer():
    answer = np.random.choice(options, 5, replace=True)
    return ''.join(answer)
random_answers = data_to_predict.TX_RESPOSTAS_MT.apply(lambda x: generate_answer())
result = (
    data_to_predict
    .assign(TX_RESPOSTAS_MT=random_answers)
    [["NU_INSCRICAO", "TX_RESPOSTAS_MT"]]
)

In [144]:
random_guys = data_to_predict[data_to_predict.TX_RESPOSTAS_MT == 'CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC'].NU_INSCRICAO
result["TX_RESPOSTAS_MT"] = np.where(result["NU_INSCRICAO"].isin(random_guys), 'CCCCC', result["TX_RESPOSTAS_MT"])

### Random using previous answers frequencies as probabilities

In [200]:
options = ['A', 'B', 'C', 'D', 'E']
def generate_prob_answer(prev_answers):
    prev_answers = np.array(list(prev_answers) + options)
    unique, counts = np.unique(prev_answers, return_counts=True)
    if '*' in unique:
        answer = np.random.choice(options, 5, replace=True)
        return ''.join(answer)
    probs = counts/np.sum(counts)
    answer = np.random.choice(options, 5, p=probs, replace=True)
    return ''.join(answer)
random_answers = data_to_predict.TX_RESPOSTAS_MT.apply(lambda x: generate_prob_answer(x))
result = (
    data_to_predict
    .assign(TX_RESPOSTAS_MT=random_answers)
    [["NU_INSCRICAO", "TX_RESPOSTAS_MT"]]
)

In [204]:
result.head()

,NU_INSCRICAO,TX_RESPOSTAS_MT
0,060edb14439f3a7b0e736493d8dc5a45de16ed51,CDAEA
1,63929f181ec794c4a94176b61f0d1def6f4799fe,BADBD
2,11c89c0cb5dc38aa74b242f702bf9df1edc23b5e,CEBCD
3,297931c1902ee1c5cd7bf2ee16b148dedd8e3a9f,EBCAB
4,14126c55c1c250f0d769526cf4d5383add31873d,ABAAE


### Generate Sub

In [202]:
answer = result.to_dict("records")
submission = {
    "token": "5cfb5e6838b5b71058949df3e8746d3ff1c31a73",
    "email": "igor.a.r.y@gmail.com",
    "answer": answer
}
with open("submission.json", "w") as fp:
    json.dump(submission, fp)

### Send Post request to codenation API

In [203]:
url = 'https://api.codenation.com.br/v1/user/acceleration/data-science/challenge/enem-3/submit'
r = requests.post(url, json=submission)
r.json()

{'code': '500',
 'error': 'read tcp 172.31.3.138:8080->172.31.6.212:7926: i/o timeout',
 'message': 'Erro adicionando o desafio'}